# Validation
## Juncal Norte - august, 2022
#### Paul Sandoval Quilodrán - https://github.com/SQPaul

## Import packages

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import re
import matplotlib.pyplot as plt
import datetime
import hydroeval as he

### Streamflow by source

In [2]:
#Define paths

rain_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QRAINSubBasinTSS.tss"
snow_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QSNOWSubBasinTSS.tss"
glacier_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QGLACSubBasinTSS.tss"
baseflow_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QBASFSubBasinTSS.tss"
obs_path = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\qobs_v1.xlsx"

paths = [rain_path, snow_path, glacier_path, baseflow_path]

rain = []
snow = []
glacier = []
baseflow = []
sim = []

streamflow = [rain,snow,glacier,baseflow]

for i in range(len(paths)):
    var = pd.read_csv(paths[i],index_col=False,skiprows=4)
    for j in range(len(var)):
        val = var.iloc[j,0].strip()
        val = " ".join(val.split())
        streamflow[i].append(val)       
    streamflow[i] = pd.DataFrame(streamflow[i])
    streamflow[i].columns = ["name"]
    streamflow[i] = streamflow[i]["name"].str.split(" ",expand=True)
    streamflow[i] = pd.DataFrame(streamflow[i].iloc[:,2])


obs = pd.read_excel(obs_path,sheet_name="5",engine="openpyxl")
        
rain = pd.DataFrame(streamflow[0]).astype(float)
rain.columns = ["rain"]
snow = pd.DataFrame(streamflow[1]).astype(float)
snow.columns = ["snow"]
glacier = pd.DataFrame(streamflow[2]).astype(float)
glacier.columns = ["glacier"]
baseflow = pd.DataFrame(streamflow[3]).astype(float)
baseflow.columns = ["baseflow"]

for k in range(len(rain)):
    sim.append(rain.iloc[k,0]+snow.iloc[k,0]+glacier.iloc[k,0]+baseflow.iloc[k,0])

sim = pd.DataFrame(sim)
sim.columns = ["sim"]
obs = pd.DataFrame(obs.iloc[:,1]).astype(float)
obs.columns = ["obs"]

streamflow = pd.concat([baseflow,rain,snow,glacier,sim,obs],axis=1)
streamflow.to_excel(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\postprocess\streamflow_validation.xlsx")

### KGE

In [3]:
#Import paths
obs_path = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\qobs_v1.xlsx"
sim_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QTOTSubBasinTSS.tss"
thickness_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\ICE_DEPTH.csv"


#Read and prepare obs streamflow
obs = pd.read_excel(obs_path,sheet_name="5",engine="openpyxl")
obs = obs.iloc[:,1]

#Read and prepare sim streamflow
sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
sim = []
for v in range(len(sim_read)):
    val = sim_read.iloc[v,0].strip()
    val = " ".join(val.split())
    sim.append(val)
sim = pd.DataFrame(sim)
sim.columns = ["name"]
sim = sim["name"].str.split(" ",expand=True)
sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
sim = sim.iloc[:,0]

#Calculate KGE'
kge = pd.DataFrame(he.evaluator(he.kgeprime, sim, obs),index=["KGE","r","γ","β"])
kge

#Read thickness
thickness = pd.read_csv(thickness_path)

g_14053 = []
g_14088 = []
g_14092 = []
g_14093 = []

for i in range(len(thickness)):
        if thickness.iloc[i,0] == "2000-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2005-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2010-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2015-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2020-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])


glacier_t = pd.DataFrame(np.column_stack([g_14053,g_14088,g_14092,g_14093]), 
                               columns=["14053","14088","14092","14093"])
glacier_t.index = ["2015","2020"]

with pd.ExcelWriter(r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Calibration\Sensitivity_analysis\sensitivity_analysis.xlsx") as writer:
    kge.to_excel(writer, sheet_name='kge')
    glacier_t.to_excel(writer, sheet_name='thickness')
    
kge

,0
KGE,0.300227
r,0.787385
γ,1.656656
β,0.884760


### NASH

In [4]:
#Import paths
obs_path = r"C:\Users\pauls\Desktop\Proyectos\Juncal_norte\Caudales\qobs_v1.xlsx"
sim_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\QTOTSubBasinTSS.tss"
thickness_path = r"C:\Users\pauls\Anaconda3\pkgs\sphy-2.2.1-py_0\site-packages\SPHY\Juncal_norte\output\ICE_DEPTH.csv"


#Read and prepare obs streamflow
obs = pd.read_excel(obs_path,sheet_name="5",engine="openpyxl")
obs = obs.iloc[:,1]

#Read and prepare sim streamflow
sim_read = pd.read_csv(sim_path,index_col=False,skiprows=4)
sim = []
for v in range(len(sim_read)):
    val = sim_read.iloc[v,0].strip()
    val = " ".join(val.split())
    sim.append(val)
sim = pd.DataFrame(sim)
sim.columns = ["name"]
sim = sim["name"].str.split(" ",expand=True)
sim = pd.DataFrame(sim.iloc[:,2]).astype(float)
sim = sim.iloc[:,0]

#Calculate KGE'
nash = pd.DataFrame(he.evaluator(he.nse, sim, obs),index=["NASH"])
nash

#Read thickness
thickness = pd.read_csv(thickness_path)

g_14053 = []
g_14088 = []
g_14092 = []
g_14093 = []

for i in range(len(thickness)):
        if thickness.iloc[i,0] == "2000-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2005-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2010-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2015-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])
        elif thickness.iloc[i,0] == "2020-01-01":
            g_14053.append(thickness.iloc[i,1])
            g_14088.append(thickness.iloc[i,2])
            g_14092.append(thickness.iloc[i,3])
            g_14093.append(thickness.iloc[i,4])

glacier_t = pd.DataFrame(np.column_stack([g_14053,g_14088,g_14092,g_14093]), 
                               columns=["14053","14088","14092","14093"])
glacier_t.index = ["2015","2020"]
    
nash

,0
NASH,0.126264


In [6]:
glacier_t

,14053,14088,14092,14093
2015,20.425615,36.455643,41.26439,7.842469
2020,14.703634,27.590780,34.67041,0.000000
